# 3-1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from category_encoders import *
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from scipy import stats
import pickle

In [ ]:
df = pd.read_csv("Week 3 Data/BUYTEST.csv")
df

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
#Converting strange null to np null
compare_str = df['AGE'][9]
for col in df.columns:
    df.loc[df[col] == compare_str, col] = np.nan

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
#Type conversion of columns
df[['AGE','INCOME','MARRIED','FICO','OWNHOME']] = df[['AGE','INCOME','MARRIED','FICO','OWNHOME']].apply(pd.to_numeric)


In [ ]:
df.dtypes

In [ ]:
#Missing Value treatment
df['AGE'].fillna(df['AGE'].mean(),inplace=True)
df['FICO'].fillna(df['FICO'].mean(),inplace=True)
df['INCOME'].fillna(df['INCOME'].mean(),inplace=True)
df['SEX'].fillna(df['SEX'].value_counts().idxmax(),inplace=True)
df['MARRIED'].fillna(df['MARRIED'].value_counts().idxmax(),inplace=True)
df['OWNHOME'].fillna(df['OWNHOME'].value_counts().idxmax(),inplace=True)
df['ORGSRC'].fillna(df['ORGSRC'].value_counts().idxmax(),inplace=True)

In [ ]:
#Categorical Encoder
enc = BinaryEncoder(cols=['SEX', 'LOC','ORGSRC']).fit(df)
enc_df = enc.transform(df)

In [ ]:
enc_df.isnull().any()

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
enc_df.drop(['ID','C1','C2','C3','C4','C5','C6','C7','PURCHTOT'],axis=1,inplace=True)

In [ ]:
#pie chart
pd.value_counts(enc_df['RESPOND']).plot.pie()

In [ ]:
#partitioning
train, test = train_test_split(enc_df,test_size=0.3)

In [ ]:
train.columns

In [ ]:
X_train = train.loc[:,enc_df.columns != 'RESPOND'].values
y_train = train.loc[:,enc_df.columns == 'RESPOND'].values
X_test = test.loc[:,enc_df.columns != 'RESPOND'].values
y_test = test.loc[:,enc_df.columns == 'RESPOND'].values

In [ ]:
sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train,y_train.ravel())

In [ ]:
classifier = DecisionTreeClassifier(criterion='entropy',random_state=0)
classifier.fit(X_res,y_res)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,y_pred)

In [ ]:
cm

In [ ]:
abs(((cm[0][1]+cm[1][0])/(cm[0][0]+cm[1][1]))-1)

In [ ]:
classifier2 = DecisionTreeClassifier(criterion='gini',random_state=0)
classifier2.fit(X_res,y_res)

In [ ]:
y_pred2 = classifier2.predict(X_test)
cm2 = confusion_matrix(y_test,y_pred2)
cm2

In [ ]:
abs(((cm2[0][1]+cm2[1][0])/(cm2[0][0]+cm2[1][1]))-1)

# 3-2

In [ ]:
df = pd.read_excel('Week 3 Data/CallsData.xls')
df2 = pd.read_csv('Week 3 Data/ContractData.csv')

In [ ]:
df2

In [ ]:
#inner join (join on index should be used by pd.join(how='inner'))
df = pd.merge(df,df2,on=['Area Code','Phone'])

In [ ]:
df.describe()

In [ ]:
#column filter
df.drop(['Phone','Account Length'],axis=1,inplace=True)

In [ ]:
#number to string(categorical)
print(df['Churn'].unique()) #its binary, so we dont have to encode this one.
print(df['Area Code'].unique())
print(df['State'].unique())

In [ ]:
enc = BinaryEncoder(cols=['Area Code','State']).fit(df)
enc_df = enc.transform(df)

In [ ]:
enc_df.describe()

In [ ]:
#normalize using z-score
enc_df = enc_df.apply(lambda x: x if np.std(x) == 0 or x.name == 'Churn' or 'Area Code' in x.name or 'State' in x.name else stats.zscore(x))

In [ ]:
#Partition
X = enc_df.loc[:,enc_df.columns != 'Churn'].values
y = enc_df.loc[:,enc_df.columns == 'Churn'].values.ravel()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
#Oversampling
sm = SMOTE()
X_train_res, y_train_res = sm.fit_resample(X_train,y_train)

In [ ]:
len(X_train_res)

In [ ]:
classifier = DecisionTreeClassifier(criterion='gini',min_samples_split=6)
classifier.fit(X_train_res,y_train_res)

In [ ]:
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
print('Accuracy:',abs(((cm[0][1]+cm[1][0])/(cm[0][0]+cm[1][1]))-1))

In [ ]:
DT_classifier = pickle.dumps(classifier)